In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from keras_tuner import HyperParameters

import autokeras as ak

In [2]:
from cerebro.nas.hphpmodel import HyperHyperModel

In [3]:
input_node = ak.ImageInput()
# output_node = ak.Normalization()(input_node)
# output_node1 = ak.ConvBlock()(output_node)
# output_node2 = ak.ResNetBlock(version="v2")(output_node)
# output_node = ak.Merge()([output_node1, output_node2])
# output_node = ak.ClassificationHead()(output_node)
output_node = ak.ClassificationHead()

am = HyperHyperModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

In [4]:
am.graph

In [5]:
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

2021-11-08 10:06:38 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
from cerebro.backend import SparkBackend
from cerebro.storage import LocalStore

backend = SparkBackend(spark_context=sc, num_workers=1)
store = LocalStore(prefix_path='/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/experiments')

am.resource_bind(backend=backend, store=store)

CEREBRO => Time: 2021-11-08 10:06:39, Running 1 Workers


In [7]:
cus_hp = HyperParameters()
cus_hp.Fixed("learning_rate", value=0.0001)
am.tuner_bind("hyperband", hyperparameters=cus_hp)

# am.tuner_bind("hyperband", hyperparameters=None)

INFO:tensorflow:Reloading Oracle from existing project ./test/oracle.json


In [8]:
tuner = am.tuner

tuner.search_space_summary()

Search space summary
Default search space size: 16
learning_rate (Fixed)
{'conditions': [], 'value': 0.0001}
conv_block_1/kernel_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
conv_block_1/separable (Boolean)
{'default': False, 'conditions': []}
conv_block_1/max_pooling (Boolean)
{'default': True, 'conditions': []}
conv_block_1/dropout (Choice)
{'default': 0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
conv_block_1/num_blocks (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
conv_block_1/num_layers (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2], 'ordered': True}
conv_block_1/filters_0_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_0_1 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_1_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 3

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train = np.array(tf.one_hot(y_train, len(np.unique(y_train))))

2021-11-08 10:07:41.342248: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000, 10)


In [11]:
am.test_tuner_space(x=x_train, y=y_train)

In [12]:
tuner.search_space_summary()

Search space summary
Default search space size: 23
learning_rate (Fixed)
{'conditions': [], 'value': 0.0001}
conv_block_1/kernel_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
conv_block_1/separable (Boolean)
{'default': False, 'conditions': []}
conv_block_1/max_pooling (Boolean)
{'default': True, 'conditions': []}
conv_block_1/dropout (Choice)
{'default': 0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
conv_block_1/num_blocks (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
conv_block_1/num_layers (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2], 'ordered': True}
conv_block_1/filters_0_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_0_1 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_1_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 3

In [13]:
trial = tuner.oracle.create_trial(tuner.tuner_id)
trial.summary()

Trial summary
Hyperparameters:
learning_rate: 0.0001
conv_block_1/kernel_size: 3
conv_block_1/separable: True
conv_block_1/max_pooling: False
conv_block_1/dropout: 0.25
conv_block_1/num_blocks: 1
conv_block_1/num_layers: 2
conv_block_1/filters_0_0: 256
conv_block_1/filters_0_1: 16
conv_block_1/filters_1_0: 256
conv_block_1/filters_1_1: 128
res_net_block_1/pretrained: True
res_net_block_1/version: resnet152_v2
res_net_block_1/imagenet_size: True
classification_head_1/dropout: 0.25
optimizer: sgd
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0


In [ ]:
AutoKeras test

In [13]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.Merge()([output_node1, output_node2])
output_node = ak.ClassificationHead()(output_node)

am = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

In [14]:
dataset, validation_data = am._convert_to_dataset(
            x=x_train, y=y_train, validation_data=None, batch_size=32
        )

TypeError: Expect the target data of classification_head_2 to be tf.data.Dataset, np.ndarray, pd.DataFrame or pd.Series, but got <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [5]:
dataset

<ZipDataset shapes: ((None, 28, 28), (None,)), types: (tf.uint8, tf.uint8)>

In [25]:
am._analyze_data(dataset)
am._build_hyper_pipeline(dataset)

2021-11-07 22:41:12.295138: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


In [7]:
tuner = am.tuner

In [26]:
tuner.hypermodel.hypermodel.set_fit_args(0.2, epochs=100)

In [32]:
hp = tuner.oracle.get_space()

pipeline = tuner.hyper_pipeline.build(hp, dataset)
pipeline.fit(dataset)
dataset = pipeline.transform(dataset)

In [33]:
dataset

<ZipDataset shapes: ((None, 28, 28), (None, 10)), types: (tf.uint8, tf.float32)>

In [29]:
from autokeras.utils import data_utils
tuner.hypermodel.hypermodel.set_io_shapes(data_utils.dataset_shape(dataset))

In [30]:
tuner.hypermodel.build(hp)
tuner.oracle.update_space(hp)

In [31]:
tuner.search_space_summary()

Search space summary
Default search space size: 16
conv_block_1/kernel_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
conv_block_1/separable (Boolean)
{'default': False, 'conditions': []}
conv_block_1/max_pooling (Boolean)
{'default': True, 'conditions': []}
conv_block_1/dropout (Choice)
{'default': 0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
conv_block_1/num_blocks (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
conv_block_1/num_layers (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2], 'ordered': True}
conv_block_1/filters_0_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_0_1 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_1_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filte

In [17]:
data_utils.dataset_shape(dataset)

(TensorShape([None, 28, 28]), TensorShape([None, 10]))